In [1]:
import syft as sy

assigning id:<UID:f679745e-44c7-4d9a-b903-a574e26e8411> type:<class 'torch.Tensor'>


### Test 1: Send a Message from a VM Client to a VM

In [2]:
bob_vm = sy.VirtualMachine(name="Bob")
bob_vm_client = bob_vm.get_client()

In [3]:
assert bob_vm.device is None

In [4]:
bob_vm_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm_client))

the message is for me!!!
VirtualMachine:Bob:<UID:ecf6cadf-fd88-40af-9bf8-2e5981776bb0>


### Test 2: Send a message from a Device Client to a Device

In [5]:
bob_phone = sy.Device(name="Bob's iPhone")
bob_phone_client = bob_phone.get_client()

In [6]:
msg = sy.ReprMessage(address=bob_phone_client)

In [7]:
bob_phone_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_phone_client))

the message is for me!!!
Device:Bob's iPhone:<UID:9ed4ac74-c7e1-403a-93d7-edb4ed6a7c9a>


### Test 2.1: Register a VM on a Device

In [8]:
bob_phone_client.register(client=bob_vm_client)

the message is for me!!!
the message is for me!!!
Updating to Device:Bob's iPhone:<UID:9ed4ac74-c7e1-403a-93d7-edb4ed6a7c9a> on note VirtualMachine:Bob:<UID:ecf6cadf-fd88-40af-9bf8-2e5981776bb0>


In [9]:
bob_phone.store

{<UID:ecf6cadf-fd88-40af-9bf8-2e5981776bb0>: <Storable:<VirtualMachineClient id:Bob>>}

### Test 2.2 Test that registration updated "address" variable of VM

In [10]:
assert bob_vm.device is not None

In [11]:
assert bob_vm_client.device is not None

### Test 3: Send a message from a Device Client to a VM

In [12]:
bob_phone_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm_client))

the message is not for me...
<UID:ecf6cadf-fd88-40af-9bf8-2e5981776bb0>
the message is for me!!!
VirtualMachine:Bob:<UID:ecf6cadf-fd88-40af-9bf8-2e5981776bb0>


### Test 4: Send a message from a Domain Client to a VM

In [13]:
import syft as sy

In [14]:
bob_domain = sy.Domain(name="Bob's Domain")
bob_domain_client = bob_domain.get_client()

In [21]:
bob_domain_client.register(client=bob_phone_client)

the message is for me!!!
the message is for me!!!
Updating to Domain:Bob's Domain:<UID:aaf4f27a-edae-4882-acc1-cce2f3416164> on note Device:Bob's iPhone:<UID:9ed4ac74-c7e1-403a-93d7-edb4ed6a7c9a>


In [19]:
assert bob_vm_client.domain is not None

AssertionError: 

In [17]:
bob_domain_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm_client))

AttributeError: 'NoneType' object has no attribute 'id'

### Test 5: Send a message from Network Client to a VM

In [17]:
bob_network = sy.Network(name="Bob's Network")
bob_network_client = bob_network.get_client()

In [18]:
bob_network_client.register(client=bob_domain_client)

the message is for me!!!
the message is for me!!!
Updating to Public(Network:<UID:87085cc7-7850-49db-90db-b862edff7973>, Domain:None)  Private(Device:None, VM:None) on note Domain:Bob's Domain:<UID:24d30fbc-bb31-44c0-a903-3ca109cff8fc>
the message is for me!!!
Updating to Public(Network:<UID:87085cc7-7850-49db-90db-b862edff7973>, Domain:None)  Private(Device:None, VM:None) on note Device:Bob's iPhone:<UID:a545399e-3c51-4b02-8c5a-03bb99c3bd31>
the message is for me!!!
Updating to Public(Network:<UID:87085cc7-7850-49db-90db-b862edff7973>, Domain:None)  Private(Device:None, VM:None) on note VirtualMachine:Bob:<UID:dc74d41a-175d-477f-a3bd-b4b91ead6f40>


In [20]:
bob_network_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm_client.address))

the message is not for me...
the message is not for me...
the message is not for me...
the message is for me!!!
VirtualMachine:Bob:<UID:dc74d41a-175d-477f-a3bd-b4b91ead6f40>


### Test 5: Use VM Client wtih Torch Tensor

In [1]:
import syft as sy
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_client()

In [2]:
import torch as th

In [3]:
x = th.tensor([1,2,3,4])

In [4]:
xp = x.send(alice_client)

the message is for me!!!


In [5]:
y = xp + xp

the message is for me!!!


In [6]:
y.get()

the old_message is for me!!!


tensor([2, 4, 6, 8])

In [7]:
alice.store

ObjectStore:{500: tensor([1, 2, 3, 4])}

In [8]:
del xp

Deleted:<syft.ast.klass.torch.TensorPointer object at 0x7fa6c041b430>
the old_message is for me!!!


In [9]:
alice.store

ObjectStore:{}

In [ ]:
ptr = alice_client.torch.zeros(10,10)

In [1]:


# import torch as th

# sy.LONG_TYPECHECK_STACK_TRACES(True)

# x = th.tensor([1,2,3,4])
# xp = x.send(alice_client)


# y = xp + xp

# y.get()

# del xp

{syft.core.nodes.common.service.repr_service.ReprMessage: <syft.core.nodes.common.service.repr_service.ReprService at 0x7fc8707dc7f0>,
 syft.core.nodes.common.service.heritage_update_service.HeritageUpdateMessage: <syft.core.nodes.common.service.heritage_update_service.HeritageUpdateService at 0x7fc8707dc910>,
 syft.core.nodes.common.service.child_node_lifecycle_service.RegisterChildNodeMessage: <syft.core.nodes.common.service.child_node_lifecycle_service.ChildNodeLifecycleService at 0x7fc8707dc820>,
 syft.core.nodes.common.action.common.ImmediateActionWithoutReply: <syft.core.nodes.common.service.obj_action_service.ImmediateObjectActionServiceWithoutReply at 0x7fc8707dc880>,
 syft.core.nodes.common.action.save_object_action.SaveObjectAction: <syft.core.nodes.common.service.obj_action_service.ImmediateObjectActionServiceWithoutReply at 0x7fc8707dc880>,
 syft.core.nodes.common.action.run_class_method_action.RunClassMethodAction: <syft.core.nodes.common.service.obj_action_service.Immedia

In [2]:
from syft.core.nodes.common.action.function_or_constructor_action import RunFunctionOrConstructorAction

In [3]:
def constructor(path, client, args):
    return_type_name = sy.lib_ast(path, return_callable=True).return_type_name
    c = sy.lib_ast(return_type_name, return_callable=True)
    pointer_type = getattr(c, c.pointer_name)
    pointer_to_result = pointer_type(location=client)
    action = RunFunctionOrConstructorAction(path=path, args=args, kwargs={}, address=alice.address, id_at_location=pointer_to_result.id_at_location)
    client.send_immediate_msg_without_reply(msg=action)
    return pointer_to_result

In [4]:
ptr = constructor(path='torch.Tensor', client=alice_client, args=(3,3))

the message is for me!!!


In [5]:
ptr.get()

the old_message is for me!!!


tensor([[9.8091e-45, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00]])

In [6]:
from syft import lib_ast

In [8]:
lib_ast.torch.Tensor()

TypeError: __call__() missing 2 required positional arguments: 'path' and 'index'

the message is for me!!!


the old_message is for me!!!


tensor([3., 3.])

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [ ]:
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    password = Column(String)

    def __repr__(self):
        return f'User {self.name}'

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
user = User(name='John Snow', password='johnspassword')
session.add(user)

print(user.id)  # None

In [ ]:
session.commit()

In [ ]:
query = session.query(User).filter_by(name='John Snow')

In [ ]:
query.count()